# FMRI DATA ANALYSIS - NARPS DATASET BY TEAM 2T6S

This notebook reproduces the analysis made by team **2T6S** for the NARPS experiment (Botvinik-Nezer & al, 2020). 

**To use this notebook :** 
- Follow the [instructions](https://github.com/Inria-Empenn/narps_open_pipelines/blob/main/INSTALL.md) to get the input data (only derivatives are needed.
- Modify the paths:
    - `dataset_dir`: directory where the ds001734 repository is stored
    - `result_dir`: directory where the intermediate and final results will be stored
- Select the number of cores on which you want to run the analysis (`nb_cores`)

In [1]:
from os.path import isfile
from narps_open.pipelines.team_2T6S import PipelineTeam2T6S

In [2]:
# Change these values if needed
dataset_dir = '/work/data/original/ds001734/'
results_dir = '/work/run/reproduced/'
nb_cores = 2

In [3]:
# Subject list (to which we will do the analysis)
subject_list = ['020', '001', '070', '013']

# Runs to use for this analysis
run_list = ['01', '02', '03', '04']

In [4]:
pipeline = PipelineTeam2T6S()
pipeline.subject_list = subject_list
pipeline.run_list = run_list
pipeline.directories.dataset_dir = dataset_dir
pipeline.directories.results_dir = results_dir
pipeline.directories.set_output_dir_with_team_id('2T6S')
pipeline.directories.set_working_dir_with_team_id('2T6S')

## Subject level analysis

In [5]:
subject_level_analysis = pipeline.get_subject_level_analysis()

stty: 'standard input': Inappropriate ioctl for device


In [6]:
subject_level_analysis.run('MultiProc', plugin_args = {'n_procs': nb_cores})

240624-07:04:03,203 nipype.workflow INFO:
	 Workflow l1_analysis settings: ['check', 'execution', 'logging', 'monitoring']
240624-07:04:03,244 nipype.workflow INFO:
	 Running in parallel.
240624-07:04:03,303 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 5 jobs ready. Free memory (GB): 13.77/13.77, Free processors: 2/2.
240624-07:04:03,380 nipype.workflow INFO:
	 [Job 0] Cached (l1_analysis.contrasts).
240624-07:04:03,415 nipype.workflow INFO:
	 [Node] Setting-up "l1_analysis.selectfiles" in "/work/run/reproduced/NARPS-2T6S-reproduced/intermediate_results/l1_analysis/_subject_id_020/selectfiles".
240624-07:04:03,427 nipype.workflow INFO:
	 [Node] Executing "selectfiles" <nipype.interfaces.io.SelectFiles>
240624-07:04:03,703 nipype.workflow INFO:
	 [Node] Finished "selectfiles", elapsed time 0.271548s.
240624-07:04:05,306 nipype.workflow INFO:
	 [Job 1] Completed (l1_analysis.selectfiles).
240624-07:04:05,311 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 6 jobs read

KeyboardInterrupt: 

## Group level analysis

In [7]:
# Get workflows for the group level analyses in the equalIndifference (ei) equalRange (er) and group comparison (gc) cases
group_level_analysis_ei, group_level_analysis_er, group_level_analysis_gc = pipeline.get_group_level_analysis()

In [ ]:
group_level_analysis_ei.run('MultiProc', plugin_args = {'n_procs': nb_cores})

240624-07:04:52,395 nipype.workflow INFO:
	 Workflow l2_analysis_equalRange_nsub_4 settings: ['check', 'execution', 'logging', 'monitoring']
240624-07:04:52,425 nipype.workflow INFO:
	 Running in parallel.
240624-07:04:52,429 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 3 jobs ready. Free memory (GB): 13.77/13.77, Free processors: 2/2.
240624-07:04:52,531 nipype.workflow INFO:
	 [Node] Setting-up "l2_analysis_equalRange_nsub_4.selectfiles_groupanalysis" in "/work/run/reproduced/NARPS-2T6S-reproduced/intermediate_results/l2_analysis_equalRange_nsub_4/_contrast_id_0002/selectfiles_groupanalysis".
240624-07:04:52,530 nipype.workflow INFO:
	 [Node] Setting-up "l2_analysis_equalRange_nsub_4.selectfiles_groupanalysis" in "/work/run/reproduced/NARPS-2T6S-reproduced/intermediate_results/l2_analysis_equalRange_nsub_4/_contrast_id_0001/selectfiles_groupanalysis".
240624-07:04:52,540 nipype.workflow INFO:
	 [Node] Executing "selectfiles_groupanalysis" <nipype.interfaces.io.SelectFiles

In [ ]:
group_level_analysis_er.run('MultiProc', plugin_args = {'n_procs': nb_cores})

In [ ]:
group_level_analysis_gc.run('MultiProc', plugin_args = {'n_procs': nb_cores})

## Check results

In [10]:
# Print the list of non existing hypotheses files
hypothesis_files = pipeline.get_hypotheses_outputs()
print([f for f in hypothesis_files if not isfile(f)])

['/work/data/reproduced/NARPS-2T6S-reproduced/l2_analysis_equalIndifference_nsub_4/_contrast_id_0002/_threshold0/spmT_0001_thr.nii', '/work/data/reproduced/NARPS-2T6S-reproduced/l2_analysis_equalIndifference_nsub_4/_contrast_id_0002/spmT_0001.nii', '/work/data/reproduced/NARPS-2T6S-reproduced/l2_analysis_equalRange_nsub_4/_contrast_id_0002/_threshold0/spmT_0001_thr.nii', '/work/data/reproduced/NARPS-2T6S-reproduced/l2_analysis_equalRange_nsub_4/_contrast_id_0002/spmT_0001.nii', '/work/data/reproduced/NARPS-2T6S-reproduced/l2_analysis_equalIndifference_nsub_4/_contrast_id_0002/_threshold0/spmT_0001_thr.nii', '/work/data/reproduced/NARPS-2T6S-reproduced/l2_analysis_equalIndifference_nsub_4/_contrast_id_0002/spmT_0001.nii', '/work/data/reproduced/NARPS-2T6S-reproduced/l2_analysis_equalRange_nsub_4/_contrast_id_0002/_threshold0/spmT_0001_thr.nii', '/work/data/reproduced/NARPS-2T6S-reproduced/l2_analysis_equalRange_nsub_4/_contrast_id_0002/spmT_0001.nii', '/work/data/reproduced/NARPS-2T6S-r